In [ ]:
import pandas as pd
import datetime as dt
import re as re
import numpy as np

In [ ]:
ddid = pd.read_excel(r"C:\Users\emmaj\Documents\NSS\Python\Projects\ddid-rasberry-pi\data\TEIS Exit Data for NSS.xlsx", sheet_name = 3)
pd.set_option('display.max_colwidth', None)
ddid.head()

In [ ]:
ddid = pd.DataFrame(ddid)
pd.set_option('display.max_colwidth', None)
ddid.head()

ddid_mvp = ddid[['Child ID', 'DOB', 'Initial IFSP Date', 'Days from I-IFSP to Exit', 'Exit Date', 'Exit Reason', 'Exit Note']]

ddid_mvp_filtered = ddid_mvp[ddid_mvp['Exit Note'].notnull()]
pd.set_option('display.max_colwidth', None)
ddid_mvp_filtered.tail()

ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]

In [ ]:
ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains('reopen[a-z]*', flags=re.I, regex=True)]

In [ ]:
ddid_mvp_filtered['Reopened Date'] = ddid_mvp_filtered['Exit Note'].str.extract(r'Reopened: (\d{2}/\d{2}/\d{4})')
ddid_mvp_filtered.head(25)

In [ ]:
ddid_mvp_filtered['rExit Date'] = ddid_mvp_filtered['Exit Note'].str.extract(r'Exit Date: (\d{2}/\d{2}/\d{4})')
ddid_mvp_filtered.head(25)

In [ ]:
ddid_mvp_filtered['reopen'] = pd.to_datetime(ddid_mvp_filtered['Reopened Date'])
ddid_mvp_filtered.head(25)

In [ ]:
ddid_mvp_filtered['exit'] = pd.to_datetime(ddid_mvp_filtered['rExit Date'])
ddid_mvp_filtered.head(25)

In [ ]:
ddid_mvp_filtered['difference'] = ddid_mvp_filtered['reopen']-ddid_mvp_filtered['exit']
ddid_mvp_filtered.head(25)

In [ ]:
for index, row in ddid_mvp_filtered.iterrows():
    if ((row['exit'] > row['Initial IFSP Date']) and
        (row['exit'] < row['Exit Date']) and
        (row['reopen'] > row['Initial IFSP Date']) and
        (row['reopen'] < row['Exit Date'])):
        diff_days = (row['reopen'] - row['exit']).days
        ddid_mvp_filtered.at[index, 'diff_days'] = diff_days
    else:
        ddid_mvp_filtered.at[index, 'diff_days'] = np.nan


In [ ]:
ddid_mvp_filtered.head(25)

In [ ]:
newexit = pd.to_datetime(ddid_mvp_filtered['Exit Note'].str.extractall('Exit Date: (\d+.\d+.\d+)')[0]).unstack()
newexit.columns = ['exit_' + str(col + 1) for col in newexit.columns]
newexit

In [ ]:
newenter = pd.to_datetime(ddid_mvp_filtered['Exit Note'].str.extractall('Reopened: (\d+.\d+.\d+)')[0]).unstack()
newenter.columns = ['reopen_'+str(col+1) for col in newenter.columns]
newenter

In [ ]:
open_exit = newexit.join(newenter)
open_exit.head(25)

In [ ]:
new = ddid_mvp_filtered.join(open_exit)
new

In [ ]:
new['diff1'] = new.exit_1-new.reopen_1
new.head(25)

In [ ]:
new['diff2'] = new.reopen_1-new.exit_2
new.head(25)

In [ ]:
new['diff3'] = new.exit_3-new.reopen_1
new.head(25)